In [1]:
import anndata as ad
import scanpy as sc

import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import torch

from persist import PERSIST, ExpressionDataset

In [2]:
adata = ad.read_h5ad('/scratch/nmq407/dvc_neurons_rat_20kc_3kvf.h5Seurat.h5ad')
adata

AnnData object with n_obs × n_vars = 20000 × 3000
    obs: 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'pool', 'hash.ID', 'treatment', 'run', 'species', 'area', 'orig.clusters', 'zhang.predictions', 'zhang.score', 'ludwig.predictions', 'ludwig.score', 'integrated_snn_res.0.1', 'integrated_snn_res.1', 'sub.cluster', 'neurotransmitter', 'cell.type', 'major.cell.type', 'nCount_rat_RNA', 'nFeature_rat_RNA', 'nCount_SCT', 'nFeature_SCT'
    var: 'features'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [3]:
sc.pp.highly_variable_genes(adata, flavor='seurat', n_top_genes=3000, inplace=True)

In [4]:
adata

AnnData object with n_obs × n_vars = 20000 × 3000
    obs: 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'pool', 'hash.ID', 'treatment', 'run', 'species', 'area', 'orig.clusters', 'zhang.predictions', 'zhang.score', 'ludwig.predictions', 'ludwig.score', 'integrated_snn_res.0.1', 'integrated_snn_res.1', 'sub.cluster', 'neurotransmitter', 'cell.type', 'major.cell.type', 'nCount_rat_RNA', 'nFeature_rat_RNA', 'nCount_SCT', 'nFeature_SCT'
    var: 'features', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [5]:
adata.var['highly_variable']

Cnksr3        True
Nmbr          True
Trdn          True
Nkain2        True
Rps11         True
              ... 
Mt-co3        True
Mt-nd4        True
Mt-cyb        True
AY172581.8    True
Tnks2.1       True
Name: highly_variable, Length: 3000, dtype: bool

In [6]:
adata = adata[:,adata.var['highly_variable']]

In [7]:
adata.obs['ludwig.predictions']

SI-TT-B10_AAGCATCAGGGAGATA_2    GABA1
SI-TT-B2_ATTCGTTCACATGGTT_2     Chat2
SI-TT-E1_TTGAACGGTTGTGCCG_2     GABA3
SI-TT-D2_ACAAAGACATAACGGG_2      Glu1
SI-TT-C9_CCTGTTGCAAGTCGTT_2     GABA1
                                ...  
SI-TT-D5_CTAACTTGTATCGTTG_2     GABA1
SI-TT-G1_GAAGCGAGTCCAAAGG_2     GABA1
SI-TT-G9_GTGAGGACAAGTGCAG_2     GABA1
SI-TT-D9_GTGATGTGTCCAAGAG_2     GABA1
SI-TT-H9_AGGGTGATCGGTGAAG_2     GABA3
Name: ludwig.predictions, Length: 20000, dtype: object

In [8]:
adata.obs['ludwig.predictions_codes'] = pd.Categorical(adata.obs['ludwig.predictions']).codes


/tmp/ipykernel_1380/3334576302.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['ludwig.predictions_codes'] = pd.Categorical(adata.obs['ludwig.predictions']).codes
/tools/anaconda/envs/nmq407/fgf1_ygg/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


In [9]:
adata

AnnData object with n_obs × n_vars = 20000 × 3000
    obs: 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'pool', 'hash.ID', 'treatment', 'run', 'species', 'area', 'orig.clusters', 'zhang.predictions', 'zhang.score', 'ludwig.predictions', 'ludwig.score', 'integrated_snn_res.0.1', 'integrated_snn_res.1', 'sub.cluster', 'neurotransmitter', 'cell.type', 'major.cell.type', 'nCount_rat_RNA', 'nFeature_rat_RNA', 'nCount_SCT', 'nFeature_SCT', 'ludwig.predictions_codes'
    var: 'features', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [10]:
adata.layers['bin'] = (adata.X>0).astype(np.float32)

In [11]:
print(adata)

AnnData object with n_obs × n_vars = 20000 × 3000
    obs: 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'pool', 'hash.ID', 'treatment', 'run', 'species', 'area', 'orig.clusters', 'zhang.predictions', 'zhang.score', 'ludwig.predictions', 'ludwig.score', 'integrated_snn_res.0.1', 'integrated_snn_res.1', 'sub.cluster', 'neurotransmitter', 'cell.type', 'major.cell.type', 'nCount_rat_RNA', 'nFeature_rat_RNA', 'nCount_SCT', 'nFeature_SCT', 'ludwig.predictions_codes'
    var: 'features', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'bin'


In [12]:
# Choose training and validation splits. 
# You may want to use a different strategy to choose these - see https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection
train_ind, val_ind = sk.model_selection.train_test_split(np.arange(adata.shape[0]), train_size=0.8)

print(f'{adata.shape[0]} total samples')
print(f'{np.size(train_ind)} in training set')
print(f'{np.size(val_ind)} in validation set')

# These are views, so they do not take up memory
adata_train = adata[train_ind,:]
adata_val = adata[val_ind,:]

20000 total samples
16000 in training set
4000 in validation set


In [ ]:
import time
# Get the start time
start_time = time.time()
print(start_time)

# Initialize the dataset for PERSIST
# Note: Here, data_train.layers['bin'] is a sparse array
# data_train.layers['bin'].A converts it to a dense array
train_dataset = ExpressionDataset(adata_train.layers['bin'], adata_train.obs['ludwig.predictions_codes'])
val_dataset = ExpressionDataset(adata_val.layers['bin'], adata_val.obs['ludwig.predictions_codes'])


# Use GPU device if available -- we highly recommend using a GPU!
device = torch.device(torch.cuda.current_device() if torch.cuda.is_available() else 'cpu')

# Number of genes to select within the current selection process.
num_genes = (32, 64, 100)
persist_results = {}

# Set up the PERSIST selector
selector = PERSIST(train_dataset,
                   val_dataset,
                   loss_fn=torch.nn.CrossEntropyLoss(),
                   device=device)
print(device)

# Coarse removal of genes
print('Starting initial elimination...')
candidates, model = selector.eliminate(target=500, max_nepochs=250)
print('Completed initial elimination.')

print('Selecting specific number of genes...')
for num in num_genes:
    inds, model = selector.select(num_genes=num, max_nepochs=250)
    persist_results[num] = inds
print('Done')

# Get the end time
end_time = time.time()
print(time.localtime(end_time))
# Calculate the execution time
execution_time = end_time - start_time

# Format the execution time in a human-readable format
minutes, seconds = divmod(execution_time, 60)
hours, minutes = divmod(minutes, 60)
formatted_time = f"{int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds"
print("Execution time:", formatted_time)

1682376402.4261408
cuda:0
Starting initial elimination...
using CrossEntropyLoss, starting with lam = 0.0001


Training epochs:   0%|          | 0/250 [00:00<?, ?it/s]

lam = 0.000100 yielded 1414 genes
Warm starting model for next iteration
next attempt is lam = 0.000445


Training epochs:   0%|          | 0/250 [00:00<?, ?it/s]

In [19]:
persist_results

{}